In [1]:
import os
os.chdir("..")
from sklearn.datasets import fetch_covtype
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
import random
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import accuracy_score, f1_score

%load_ext autoreload
%autoreload 2

# Utility Functions

In [2]:

def make_mixed_classification(n_samples, n_features, n_categories):
    X,y = make_classification(n_samples=n_samples, n_features=n_features, random_state=42, n_informative=5)
    cat_cols = random.choices(list(range(X.shape[-1])),k=n_categories)
    num_cols = [i for i in range(X.shape[-1]) if i not in cat_cols]
    for col in cat_cols:
        X[:,col] = pd.qcut(X[:,col], q=4).codes.astype(int)
    col_names = [] 
    num_col_names=[]
    cat_col_names=[]
    for i in range(X.shape[-1]):
        if i in cat_cols:
            col_names.append(f"cat_col_{i}")
            cat_col_names.append(f"cat_col_{i}")
        if i in num_cols:
            col_names.append(f"num_col_{i}")
            num_col_names.append(f"num_col_{i}")
    X = pd.DataFrame(X, columns=col_names)
    y = pd.Series(y, name="target")
    data = X.join(y)
    return data, cat_col_names, num_col_names
    
def load_classification_data():
    dataset = fetch_covtype(data_home="data")
    data = np.hstack([dataset.data, dataset.target.reshape(-1, 1)])
    col_names = [f"feature_{i}" for i in range(data.shape[-1])]
    col_names[-1] = "target"
    data = pd.DataFrame(data, columns=col_names)
    data["feature_0_cat"] = pd.qcut(data["feature_0"], q=4)
    data["feature_0_cat"] = "feature_0_" + data.feature_0_cat.cat.codes.astype(str)
    test_idx = data.sample(int(0.2 * len(data)), random_state=42).index
    test = data[data.index.isin(test_idx)]
    train = data[~data.index.isin(test_idx)]
    return (train, test, ["target"])

def print_metrics(y_true, y_pred, tag):
    if isinstance(y_true, pd.DataFrame) or isinstance(y_true, pd.Series):
        y_true = y_true.values
    if isinstance(y_pred, pd.DataFrame) or isinstance(y_pred, pd.Series):
        y_pred = y_pred.values
    if y_true.ndim>1:
        y_true=y_true.ravel()
    if y_pred.ndim>1:
        y_pred=y_pred.ravel()
    val_acc = accuracy_score(y_true, y_pred)
    val_f1 = f1_score(y_true, y_pred)
    print(f"{tag} Acc: {val_acc} | {tag} F1: {val_f1}")

# Generate Synthetic Data 

First of all, let's create a synthetic data which is a mix of numerical and categorical features

In [3]:
data, cat_col_names, num_col_names = make_mixed_classification(n_samples=10000, n_features=20, n_categories=4)
train, test = train_test_split(data, random_state=42)
train, val = train_test_split(train, random_state=42)

## Baseline

Let's use the default LightGBM model as a baseline.

In [4]:
clf = lgb.LGBMClassifier(random_state=42)
clf.fit(train.drop(columns='target'), train['target'], categorical_feature=cat_col_names)
val_pred = clf.predict(val.drop(columns='target'))
print_metrics(val['target'], val_pred, "Validation")
test_pred = clf.predict(test.drop(columns='target'))
print_metrics(test['target'], test_pred, "Holdout")

/home/fonnesbeck/anaconda3/envs/pitch_effect/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


Validation Acc: 0.9328 | Validation F1: 0.9322580645161291
Holdout Acc: 0.9328 | Holdout F1: 0.9330677290836654


# Importing the Library

In [18]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, NodeConfig, TabNetModelConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig
from pytorch_tabular.categorical_encoders import CategoricalEmbeddingTransformer



## Define the Configs

This is the most crucial step in the process. There are four configs that you need to provide(most of them have intelligent default values), which will drive the rest of the process.

* DataConfig - Define the target column names, categorical and numerical column names, any transformation you need to do, etc.
* ModelConfig - There is a specific config for each of the models. This determines which model we are going to train and also lets you define the hyperparameters of the model
* TrainerConfig - This let's you configure the training process by setting things like batch_size, epochs, early stopping, etc. The vast majority of parameters are directly borrowed from PyTorch Lightning and is passed to the underlying Trainer object during training
* OptimizerConfig - This let's you define and use different Optimizers and LearningRate Schedulers. Standard PyTorch Optimizers and Learning RateSchedulers are supported. For custom optimizers, you can use the parameter in the fit method to overwrite this. The custom optimizer should be PyTorch compatible
* ExperimentConfig - This is an optional parameter. If set, this defines the Experiment Tracking. Right now, only two experiment tracking frameworks are supported: Tensorboard and Weights&Biases. W&B experiment tracker has more features like tracking the gradients and logits across epochs.

In [11]:
data_config = DataConfig(
    target=['target'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform="quantile_normal",
    normalize_continuous_features=True
)
trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=1024,
    max_epochs=1000,
    auto_select_gpus=False,
    gpus=0, #index of the GPU to use. 0, means CPU
)
optimizer_config = OptimizerConfig()
model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="4096-4096-512",  # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    learning_rate = 1e-3,
    metrics=["accuracy"]
)
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

## Training the Model 
Now that we have defined the configs and the TabularModel. We just need to call the `fit` method and pass the train and test dataframes. We can also pass in validation dataframe. But if omitted, TabularModel will separate 20% of the data as validation.

In [12]:
tabular_model.fit(train=train, test=test)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name                   | Type                | Params
---------------------------------------------------------------
0 | embedding_layers       | ModuleList          | 45    
1 | normalizing_batch_norm | BatchNorm1d         | 34    
2 | backbone               | FeedForwardBackbone | 19.0 M
3 | output_layer           | Linear              | 1.0 K 
4 | loss                   | CrossEntropyLoss    | 0     
/home/fonnesbeck/anaconda3/envs/pitch_effect/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/home/fonnesbeck/anaconda3/envs/pitch_effect/lib/python3.9/site-packages/pytorch_lightning/utili

Epoch 32: 100%|██████████| 7/7 [00:35<00:00,  7.09s/it, loss=0.552, train_loss=0.59, valid_loss=0.423, valid_accuracy=0.831, train_accuracy=0.698]

## Evaluating the Model
To evaluate the model on new data on the same metrics/loss that was used during training, we can use the `evaluate` method

In [13]:
result = tabular_model.evaluate(test)
print(result)

/home/fonnesbeck/anaconda3/envs/pitch_effect/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Testing: 100%|██████████| 3/3 [00:15<00:00,  4.14s/it]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': tensor(0.8416),
 'train_accuracy': tensor(0.7048),
 'train_loss': tensor(0.5903),
 'valid_accuracy': tensor(0.8311),
 'valid_loss': tensor(0.4232)}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 3/3 [00:15<00:00,  5.22s/it]
[{'train_loss': 0.5903493762016296, 'valid_loss': 0.4231547713279724, 'valid_accuracy': 0.8311111330986023, 'train_accuracy': 0.7048248052597046, 'test_accuracy': 0.8416000008583069}]


To get the prediction as a dataframe, we can use the `predict` method. This will add predictions to the same dataframe that was passed in. For classification problems, we get both the probabilities and the final prediction taking 0.5 as the threshold

In [14]:
pred_df = tabular_model.predict(test)
pred_df.head()

Generating Predictions...: 100%|██████████| 3/3 [00:14<00:00,  4.92s/it]


,num_col_0,cat_col_1,num_col_2,num_col_3,num_col_4,num_col_5,num_col_6,num_col_7,cat_col_8,num_col_9,...,num_col_14,num_col_15,num_col_16,num_col_17,num_col_18,num_col_19,target,0_probability,1_probability,prediction
6252,-2.790932,0.0,-2.010758,3.205420,-0.356361,-0.744417,0.427836,-1.492040,0.0,1.364186,...,-0.660336,-0.705788,0.229519,0.060878,-0.464394,2.879481,0,0.145202,0.854798,1
4684,-0.139585,0.0,-1.207160,2.690514,1.072764,-3.499028,1.561682,0.953991,2.0,1.243788,...,-2.726836,0.944248,0.821184,0.368647,-1.199147,0.126323,1,0.517947,0.482053,0
1731,0.001421,1.0,-0.279572,0.363639,0.852329,0.089246,0.084824,0.194984,0.0,2.668561,...,-0.508633,0.508788,-0.097083,-0.128070,-0.282642,-0.190155,0,0.830036,0.169964,0
4742,0.086662,3.0,0.798527,0.916448,-1.085978,0.512223,-0.903704,1.538725,2.0,1.518521,...,0.326685,1.343219,-1.147619,1.795053,0.857619,0.532915,1,0.469329,0.530671,1
4521,0.982186,2.0,-0.117476,-0.168583,-0.088413,-0.206658,-1.233511,-0.137569,3.0,-1.678887,...,-0.282845,0.458761,1.381926,-0.566849,-0.475947,-0.400418,1,0.269307,0.730693,1


In [15]:
print_metrics(test['target'], pred_df["prediction"], tag="Holdout")

Holdout Acc: 0.8416 | Holdout F1: 0.8558951965065502


## Extract the Learned Embedding

For the models that support (CategoryEmbeddingModel and CategoryEmbeddingNODE), we can extract the learned embeddings into a sci-kit learn style Transformer.

In [19]:
transformer = CategoricalEmbeddingTransformer(tabular_model)
transf_train = transformer.fit_transform(train)
clf = lgb.LGBMClassifier(random_state=42)
clf.fit(transf_train.drop(columns='target'), transf_train['target'])

Encoding the data...: 100%|██████████| 3/3 [00:00<00:00, 32.60it/s]


LGBMClassifier(random_state=42)

In [20]:
transf_val = transformer.transform(val)
val_pred = clf.predict(transf_val.drop(columns='target'))
print_metrics(transf_val['target'], val_pred, "Validation")
transf_test = transformer.transform(test)
test_pred = clf.predict(transf_test.drop(columns='target'))
print_metrics(transf_test['target'], test_pred, "Holdout")

Encoding the data...: 100%|██████████| 3/3 [00:00<00:00, 56.56it/s]


Validation Acc: 0.9264 | Validation F1: 0.925646551724138


Encoding the data...: 100%|██████████| 3/3 [00:00<00:00, 49.19it/s]


Holdout Acc: 0.934 | Holdout F1: 0.934445768772348


## NODE

In [21]:
OptimizerConfig?

Init signature:
OptimizerConfig(
    optimizer: str = 'Adam',
    optimizer_params: dict = <factory>,
    lr_scheduler: Optional[str] = None,
    lr_scheduler_params: Optional[dict] = <factory>,
    lr_scheduler_monitor_metric: Optional[str] = 'valid_loss',
) -> None
Docstring:     
Optimizer and Learning Rate Scheduler configuration.
Args:
    optimizer (str): Any of the standard optimizers from 
        [torch.optim](https://pytorch.org/docs/stable/optim.html#algorithms). Defaults to `Adam`"

    optimizer_params (dict): The parameters for the optimizer. If left blank, will use default parameters.

    lr_scheduler (Optional[str, NoneType]): The name of the LearningRateScheduler to use, if any, from [torch.optim.lr_scheduler](https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate). 
        If None, will not use any scheduler. Defaults to `None`

    lr_scheduler_params (Optional[dict, NoneType]): The parameters for the LearningRateScheduler. If left blank, will use d

In [29]:
data_config = DataConfig(
    target=['target'],
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform="quantile_normal",#"yeo-johnson",
    normalize_continuous_features=True
)
trainer_config = TrainerConfig(
    auto_lr_find=True,
    batch_size=128,
    max_epochs=100,
    auto_select_gpus=False,
    gpus=0,
    # track_grad_norm=2,
    gradient_clip_val=10,
)
# experiment_config = ExperimentConfig(project_name="Tabular_test", log_logits=True)
optimizer_config = OptimizerConfig()

model_config = NodeConfig(
    task="classification",
    num_layers=2,
    num_trees=1024,
    learning_rate=1,
    embed_categorical=False,
    metrics=["accuracy","f1"],
    # target_range=(train['block_0'].min().item(), train['block_0'].max().item())
)
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

In [30]:
tabular_model.fit(train=train, test=test)

/home/fonnesbeck/anaconda3/envs/pitch_effect/lib/python3.9/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/home/fonnesbeck/anaconda3/envs/pitch_effect/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Checkpoint directory saved_models exists and is not empty. With save_top_k=1, all files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name            | Type             | Params
-----------------------------------------------------
0 | backbone        | NODEBackbone     | 32.4 M
1 | output_response | Lambda           | 0     
2 | loss            | CrossEntropyLoss | 0     
/home/fonnesbeck/anaconda3/envs/pitch_effect/lib/python3.9/site-packages/pytorch_lightning/utilit

Epoch 19: 100%|██████████| 45/45 [18:35<00:00, 61.95s/it, loss=0.394, train_loss=0.232, valid_loss=0.448, valid_accuracy=0.795, valid_f1=0.795, train_accuracy=0.818, train_f1=0.818]

In [31]:
result = tabular_model.evaluate(test)
print(result)

/home/fonnesbeck/anaconda3/envs/pitch_effect/lib/python3.9/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Testing: 100%|██████████| 20/20 [07:23<00:00, 22.04s/it]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': tensor(0.8384),
 'test_f1': tensor(0.8384),
 'train_accuracy': tensor(0.8136),
 'train_f1': tensor(0.8136),
 'train_loss': tensor(0.2316),
 'valid_accuracy': tensor(0.7947),
 'valid_f1': tensor(0.7947),
 'valid_loss': tensor(0.4481)}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 20/20 [07:23<00:00, 22.18s/it]
[{'train_loss': 0.231593519449234, 'valid_loss': 0.44811415672302246, 'valid_accuracy': 0.7946666479110718, 'valid_f1': 0.7946666479110718, 'train_accuracy': 0.8135850429534912, 'train_f1': 0.8135850429534912, 'test_accuracy': 0.8384000062942505, 'test_f1': 0.8384000062942505}]


In [32]:
node_pred_df = tabular_model.predict(test)

Generating Predictions...: 100%|██████████| 20/20 [07:17<00:00, 21.85s/it]


In [33]:
print_metrics(test['target'], node_pred_df["prediction"], tag="Holdout")

Holdout Acc: 0.8384 | Holdout F1: 0.8382706164931946


## NODE (Cat Embed)

In [6]:
data_config = DataConfig(
    target=['target'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform="quantile_normal",
    normalize_continuous_features=True
)

trainer_config = TrainerConfig(
    auto_lr_find=True,
    batch_size=64,
    max_epochs=1000,
    gpus=1,
    # track_grad_norm=2,
#     gradient_clip_val=10,
)
# experiment_config = ExperimentConfig(project_name="Tabular_test", log_logits=True)
optimizer_config = OptimizerConfig()

model_config = NodeConfig(
    task="classification",
    num_layers=2,
    num_trees=512,
    learning_rate=1,
    embed_categorical=True,
    # metrics=["MeanSquaredLogError"],
    # target_range=(train['block_0'].min().item(), train['block_0'].max().item())
)
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

In [7]:
tabular_model.fit(train=train, test=test)

D:\miniconda3\envs\df_encoder\lib\site-packages\pytorch_lightning\utilities\distributed.py:45: UserWarning: Checkpoint directory saved_models exists and is not empty. With save_top_k=1, all files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)
GPU available: True, used: False
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
TPU available: False, using: 0 TPU cores
D:\miniconda3\envs\df_encoder\lib\site-packages\pytorch_lightning\utilities\distributed.py:45: UserWarning: GPU available but not used. Set the --gpus flag when calling the script.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type             | Params
-------------------------------------------------------
0 | embedd

Epoch 1:  80%|▊| 71/89 [00:04<00:01, 10.39it/s, loss=0.655, train_loss=0.63, valid_loss=0.69, valid_accuracy=0.485, tra
Epoch 1:  84%|▊| 75/89 [00:04<00:01, 11.15it/s, loss=0.655, train_loss=0.63, valid_loss=0.69, valid_accuracy=0.485, tra
Epoch 1:  90%|▉| 80/89 [00:04<00:00, 11.99it/s, loss=0.655, train_loss=0.63, valid_loss=0.69, valid_accuracy=0.485, tra
Epoch 1:  96%|▉| 85/89 [00:04<00:00, 12.78it/s, loss=0.655, train_loss=0.63, valid_loss=0.69, valid_accuracy=0.485, tra
Epoch 1: 100%|█| 89/89 [00:04<00:00, 12.39it/s, loss=0.655, train_loss=0.63, valid_loss=0.731, valid_accuracy=0.486, tr
Epoch 2:  80%|▊| 71/89 [00:04<00:01, 10.44it/s, loss=0.631, train_loss=0.603, valid_loss=0.731, valid_accuracy=0.486, t
Epoch 2:  84%|▊| 75/89 [00:04<00:01, 11.20it/s, loss=0.631, train_loss=0.603, valid_loss=0.731, valid_accuracy=0.486, t
Epoch 2:  90%|▉| 80/89 [00:04<00:00, 12.04it/s, loss=0.631, train_loss=0.603, valid_loss=0.731, valid_accuracy=0.486, t
Epoch 2:  96%|▉| 85/89 [00:04<00:00, 12.

Finding best initial lr: 100%|███████████████████████████████████████████████████████| 100/100 [00:20<00:00, 17.51it/s]

Epoch 3:  80%|▊| 71/89 [00:04<00:01, 10.38it/s, loss=0.635, train_loss=0.833, valid_loss=0.668, valid_accuracy=0.588, t
Epoch 3:  84%|▊| 75/89 [00:04<00:01, 11.14it/s, loss=0.635, train_loss=0.833, valid_loss=0.668, valid_accuracy=0.588, t
Epoch 3:  90%|▉| 80/89 [00:04<00:00, 11.98it/s, loss=0.635, train_loss=0.833, valid_loss=0.668, valid_accuracy=0.588, t
Epoch 3:  96%|▉| 85/89 [00:04<00:00, 12.77it/s, loss=0.635, train_loss=0.833, valid_loss=0.668, valid_accuracy=0.588, t
Epoch 3: 100%|█| 89/89 [00:04<00:00, 13.30it/s, loss=0.635, train_loss=0.833, valid_loss=0.798, valid_accuracy=0.548, t
Epoch 4:  80%|▊| 71/89 [00:04<00:01, 10.35it/s, loss=0.606, train_loss=0.516, valid_loss=0.798, valid_accuracy=0.548, t
Epoch 4:  84%|▊| 75/89 [00:04<00:01, 11.10it/s, loss=0.606, train_loss=0.516, valid_loss=0.798, valid_accuracy=0.548, t
Epoch 4:  90%|▉| 80/89 [00:04<00:00, 11.94it/s, loss=0.606, train_loss=0.516, valid_loss=0.798, valid_accuracy=0.548, t
Epoch 4:  96%|▉| 85/89 [00:04<00:00, 12.

In [ ]:
result = tabular_model.evaluate(test)
print(result)

In [ ]:
cat_embed_node_pred_df = tabular_model.predict(test)

In [ ]:
print_metrics(test['target'], cat_embed_node_pred_df["prediction"], tag="Holdout")

### Use Category embedding

In [25]:
transformer = CategoricalEmbeddingTransformer(tabular_model)
transf_train = transformer.fit_transform(train)
clf = lgb.LGBMClassifier(random_state=42)
clf.fit(transf_train.drop(columns='target'), transf_train['target'])

LGBMClassifier(random_state=42)

In [26]:
transf_val = transformer.transform(val)
val_pred = clf.predict(transf_val.drop(columns='target'))
val_acc = accuracy_score(transf_val['target'].values.ravel(), val_pred)
val_f1 = f1_score(transf_val['target'].values.ravel(), val_pred)
print(f"Val Acc: {val_acc} | Val F1: {val_f1}")

Val Acc: 0.9322666666666667 | Val F1: 0.932410856838744


In [27]:
transf_test = transformer.transform(test)
test_pred = clf.predict(transf_test.drop(columns='target'))
test_acc = accuracy_score(transf_test['target'].values.ravel(), test_pred)
test_f1 = f1_score(transf_test['target'].values.ravel(), test_pred)
print(f"Test Acc: {test_acc} | Test F1: {test_f1}")

Test Acc: 0.9368 | Test F1: 0.9383294301327089


In [24]:
## TabNet

In [25]:
data_config = DataConfig(
    target=['target'],
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    continuous_feature_transform=None,#"yeo-johnson",
    normalize_continuous_features=True
)
trainer_config = TrainerConfig(
    auto_lr_find=False,
    batch_size=1024,
    max_epochs=1000,
    gpus=1,
    # track_grad_norm=2,
    gradient_clip_val=10,
)
# experiment_config = ExperimentConfig(project_name="Tabular_test", log_logits=True)
optimizer_config = OptimizerConfig()

model_config = TabNetModelConfig(
    task="classification",
    n_d=5,
    n_a=5,
    n_steps=2,
    n_independent=2,
    n_shared=2,
    learning_rate=1e-3
)
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

7/7 [00:00<00:00, 19.07it/s, loss=0.613, train_loss=0.605, valid_loss=0.612, valid_accuracy=0.676, train_accuracy=0.672]
Epoch 84: 100%|██████████| 7/7 [00:00<00:00, 18.98it/s, loss=0.328, train_loss=0.328, valid_loss=0.336, valid_accuracy=0.877, train_accuracy=0.865]


In [26]:
tabular_model.fit(train=train, test=test)

Testing: 100%|██████████| 3/3 [00:00<00:00, 27.85it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_accuracy': tensor(0.8600, device='cuda:0'),
 'train_accuracy': tensor(0.8683, device='cuda:0'),
 'train_loss': tensor(0.3278, device='cuda:0'),
 'valid_accuracy': tensor(0.8773, device='cuda:0'),
 'valid_loss': tensor(0.3365, device='cuda:0')}
--------------------------------------------------------------------------------
Testing: 100%|██████████| 3/3 [00:00<00:00, 23.87it/s]
[{'train_loss': 0.3278144598007202, 'valid_loss': 0.3364557921886444, 'valid_accuracy': 0.8773333430290222, 'train_accuracy': 0.8682800531387329, 'test_accuracy': 0.8600000143051147}]


In [27]:
result = tabular_model.evaluate(test)
print(result)